In this jupyter notebook you can find a data cleaning project implemented in a data set that contains information about Shark Attacks.

This has been made trough a combination of data wrangling, cleaning, and manipulation with Pandas, Numpy and Regrex libraries.

All of the cleaning process below are implemented in order to demostrate wheter the next hypothesis are true:

    H1: Women who are attacked by sharks are more likely to die than men. 
    H2: In North America, most of the people attacked were surfing.
    H3: In 2018, persons in theirs 30s were the ones who experimented the bigger amount of attacks.
   

# 1. Import Numpy, Pandas and  Redex alias them to `np` and `pd` respectively.

In [1]:
# This are the libraries used in this jupyter:
import pandas as pd
import numpy as np
from numpy import nan
import re

# Importing the cleaning functions:
import src.cleaning_functions as cf

# 2. We are going to import out dataframe.

In [2]:
df = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1") #To load the data csv

# 3. EDA

### 3.1 Exploring the dataframe:

In [3]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [4]:
df.shape

(25723, 24)

In [5]:
df.keys()

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [6]:
#Let´s check some of the unique values:
df["Unnamed: 22"].unique()
df["Unnamed: 23"].unique()
df["href formula"].unique()
df["Case Number.1"].unique()
df["Case Number.2"].unique()
df["original order"].unique()
df["Investigator or Source"].unique()

array(['R. Collier, GSAF', 'K.McMurray, TrackingSharks.com',
       'B. Myatt, GSAF', ..., 'F. Schwartz, p.23; C. Creswell, GSAF',
       'The Sun, 10/20/1938', 'S.W. Baker'], dtype=object)

Thanks to the last step, we have verified that within our initial data frame there are several columns that we will not need to confirm the hypotheses. Therefore, to make the data analysis smoother we are going to eliminate them, generating a new dataframe with only those columns that are really useful for this particular project.

In [7]:
df.drop(["Date", "Species ", "Area", "Location", "Time", "Type", 
         "Name", "Injury","href", "Unnamed: 22", "Unnamed: 23",
         "href formula", "pdf", "Case Number", "Case Number.1",
         "Case Number.2", "original order", "Investigator or Source"], axis=1, inplace=True)

In [8]:
df.shape

(25723, 6)

In [9]:
# Let´s check how many nan values do we have in our new dataframe:
df.isna().sum()

Year           19423
Country        19471
Activity       19965
Sex            19986
Age            22252
Fatal (Y/N)    19960
dtype: int64

### 3.2 Drop of duplicate values

In [10]:
# Point out that from this point, out data frame is called df2
df2 = df.drop_duplicates()

In [11]:
# Data after droping duplicate values:
print(df2.shape)
print(df2.keys())

(6002, 6)
Index(['Year', 'Country', 'Activity', 'Sex ', 'Age', 'Fatal (Y/N)'], dtype='object')


In [12]:
# It is also interesting to check de type of our data:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6002 entries, 0 to 6302
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         5999 non-null   float64
 1   Country      5953 non-null   object 
 2   Activity     5482 non-null   object 
 3   Sex          5461 non-null   object 
 4   Age          3373 non-null   object 
 5   Fatal (Y/N)  5468 non-null   object 
dtypes: float64(1), object(5)
memory usage: 328.2+ KB


### 3.3 Rename  Columns of df2

In [13]:
columns = list(df2.columns)

In [14]:
new_columns = {column: column.replace(" ", "") for column in columns}
new_columns
df2.rename(columns = new_columns, inplace=True)
df2.head()

/Users/franciscovispo/opt/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Year,Country,Activity,Sex,Age,Fatal(Y/N)
0,2018.0,USA,Paddling,F,57,N
1,2018.0,USA,Standing,F,11,N
2,2018.0,USA,Surfing,M,48,N
3,2018.0,AUSTRALIA,Surfing,M,NaN,N
4,2018.0,MEXICO,Free diving,M,NaN,N


# 4. Cleaning Variables

## 4.1 Through str.methods( ):

### 4.1.1 Cleaning variable "SEX"

In [15]:
# First we are going to look at the unique values that are under this variable:
list(df2["Sex"].unique())

#Now we use the string method replace to clean the variable:
df2["Sex"] = df2["Sex"].str.replace("lli", "Unknown")
df2["Sex"] = df2["Sex"].str.replace(".", "Unknown")
df2["Sex"] = df2["Sex"].str.replace("N", "Unknown")
df2["Sex"] = df2["Sex"].str.replace("M ", "M")

<ipython-input-15-1e49d998fdae>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Sex"] = df2["Sex"].str.replace("lli", "Unknown")
<ipython-input-15-1e49d998fdae>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2["Sex"] = df2["Sex"].str.replace(".", "Unknown")
<ipython-input-15-1e49d998fdae>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["

### 4.1.2 Cleaning variable "FATAL Y/N"

In [16]:
# First we are going to look at the unique values that are under this variable:
list(df2["Fatal(Y/N)"].unique())

#Now we use the string method replace to clean the variable:
df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace(" N", "N")
df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace("N ", "N")
df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace("y", "Y")
df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace("M", "UNKNOWN")
df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace("2017", "UNKNOWN")

<ipython-input-16-7934303ede50>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace(" N", "N")
<ipython-input-16-7934303ede50>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace("N ", "N")
<ipython-input-16-7934303ede50>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

## 4.2 Through Regrex &/or Functions :

### 4.2.1 Cleaning Variable Activity

In [17]:
# First we are going to look at the unique values that are under this variable:
list(df2["Activity"].unique())

# As there are nan values on it, we are going to replace those with "Unknown", so we can cathegorize them:
df2["Activity"] = df2["Activity"].fillna("Unknown")

# We apply the function clean_activity to our variable Activity:
df2["Activity"] = df2["Activity"].apply(cf.clean_activity)

# We visualize the final values after applying the cleaning function.
print(df2["Activity"].unique())

<ipython-input-17-517379c24a22>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Activity"] = df2["Activity"].fillna("Unknown")


['Other' 'Surf' 'Free Swimming/Diving' 'Fishing' 'Interaction with sharks'
 'Water Sports' 'Sea Disaster' 'Felt into the sea' 'Sailing' 'Plane Crash']


<ipython-input-17-517379c24a22>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Activity"] = df2["Activity"].apply(cf.clean_activity)


### 4.2.2 Cleaning Variable Country:

In [18]:
# First we are going to look at the unique values that are under this variable:
df2["Country"].unique()

# As there are nan values on it, we are going to replace those with "Unknown", so we can cathegorize them:
df2["Country"] = df2["Country"].fillna("Unknown")

# We apply the function clean_activity to our variable Activity:
df2["Country"] = df2["Country"].apply(cf.clean_country)

# We visualize the final values after applying the cleaning function.
print(df2["Country"].unique())

#As now we are dealing with continents, we change the column name from Country to Continent:
new_column_continent = {column: column.replace("Country","Continent") for column in columns}
df2.rename(columns = new_column_continent, inplace=True)

['North America' 'Oceania' 'South America' 'Unknown' 'Africa' 'Asia'
 'Europe']


<ipython-input-18-4d0a8e24131e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Country"] = df2["Country"].fillna("Unknown")
<ipython-input-18-4d0a8e24131e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Country"] = df2["Country"].apply(cf.clean_country)
/Users/franciscovispo/opt/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

### 4.2.3 Cleaning Variable Year

In [19]:
# We apply the function clean_year to our variable Year:
df2["Year"] = df2["Year"].apply(cf.clean_year)

<ipython-input-19-f0c828877665>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Year"] = df2["Year"].apply(cf.clean_year)


### 4.2.4 Cleaning variable Age

In [20]:
# First we are going to look at the unique values that are under this variable:
df2["Age"].unique()

# We apply the function clean_activity to our variable Activity:
df2["Age"] = df2["Age"].apply(cf.clean_age)

#Now we use regex trough a str.replace so we can remove the '' data point:

df2["Age"] = df2["Age"].replace(r'^\s*$', np.nan, regex=True)

<ipython-input-20-a705dabff960>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Age"] = df2["Age"].apply(cf.clean_age)
<ipython-input-20-a705dabff960>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Age"] = df2["Age"].replace(r'^\s*$', np.nan, regex=True)


In [21]:
#In this variable we are not going to remove the nan values, as I do not want to bias the latter 
#visualizations around the mean or median. Thus, this cell just show us the sum of nan values for this variable
df2["Age"].isna().sum()

2635

# 5. Download clean data into a new csv. file

In [22]:
df2.to_csv("data/clean_attacks.csv")